In [4]:
!python -V

Python 3.10.6


In [5]:
import numpy as np
import pandas as pd
from src.data.read import load_file_from_s3
from src.data.read import ID_COLUMN
from src.models import predict

In [6]:
predict.get_model()

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 21cf301e4e7f459bae86218626159897

In [7]:
from src.clients import remote_client

predict_client = remote_client.PredictLambdaClient('http://localhost:9000')


In [8]:
passenger_X = {"Age": 23, "Sex": 'female', "Pclass": 2, "Embarked": 'C', "SibSp": 1, "Parch": 1, "Fare": 9}

predict_client.lambda_predict(passenger_X)

'[{"PassengerId": -1, "Prediction": 0.8866254687309265, "Survived": 1}]'

In [9]:
lambda_json_s3_path = "s3://mlflow-enkidupal-experiments/data/external/test/test.csv"

response = predict_client.predict_s3path(lambda_json_s3_path)
response


'[{"PassengerId": 892, "Prediction": 0.17178867757320404, "Survived": 0}, {"PassengerId": 893, "Prediction": 0.4923032522201538, "Survived": 0}, {"PassengerId": 894, "Prediction": 0.1256335973739624, "Survived": 0}, {"PassengerId": 895, "Prediction": 0.1516686975955963, "Survived": 0}, {"PassengerId": 896, "Prediction": 0.4442468285560608, "Survived": 0}, {"PassengerId": 897, "Prediction": 0.1707412302494049, "Survived": 0}, {"PassengerId": 898, "Prediction": 0.7031752467155457, "Survived": 1}, {"PassengerId": 899, "Prediction": 0.2966756820678711, "Survived": 0}, {"PassengerId": 900, "Prediction": 0.6736209392547607, "Survived": 1}, {"PassengerId": 901, "Prediction": 0.10040461272001266, "Survived": 0}, {"PassengerId": 902, "Prediction": 0.1366390585899353, "Survived": 0}, {"PassengerId": 903, "Prediction": 0.25172755122184753, "Survived": 0}, {"PassengerId": 904, "Prediction": 0.9200294017791748, "Survived": 1}, {"PassengerId": 905, "Prediction": 0.1651614010334015, "Survived": 0}, {

In [9]:
df2 = pd.read_json(response)
df2

,PassengerId,predictions,decisions
0,892,0.171789,0
1,893,0.492303,0
2,894,0.125634,0
3,895,0.151669,0
4,896,0.444247,0
...,...,...,...
413,1305,0.136639,0
414,1306,0.956732,1
415,1307,0.126338,0
416,1308,0.136639,0


In [8]:
passenger_X = {"Age": 23, "Sex": 'female', "Pclass": 2, "Embarked": 'C', "SibSp": 1, "Parch": 1, "Fare": 9}

test_df = predict.create_features(passenger_X)

test_df


,Age,Sex,Pclass,Embarked,SibSp,Parch,Fare
0,23,female,2,C,1,1,9


In [12]:

DEFAULT_TARGET = 'Survived'
DEFAULT_CATEGORICAL = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
DEFAULT_NUMERICAL = ['Fare', 'Age']
ID_COLUMN = 'PassengerId'


DEFAULT_ALL_COLUMNS = [ID_COLUMN] + DEFAULT_CATEGORICAL + DEFAULT_NUMERICAL
DEFAULT_TRAIN_ALL_COLUMNS = DEFAULT_CATEGORICAL + DEFAULT_NUMERICAL

DEFAULT_ALL_COLUMNS

['PassengerId', 'Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch', 'Fare', 'Age']

### Make csv file submission for Kaggle using kaggle_submission_client

In [10]:
from src.clients import local_client
lambda_json_s3_path = "s3://mlflow-enkidupal-experiments/data/external/test/test.csv"

kaggle_submission_client = local_client.LocalKaggleSubmissionClient(local_client=local_client.LocalPredictClient())

submission = kaggle_submission_client.make_kaggle_submission_s3_path(lambda_json_s3_path)
submission

with open('result.csv', 'w') as fp:
    fp.write(submission)


mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: 21cf301e4e7f459bae86218626159897

